# ⏳ Time Series Forecasting with LSTM

This notebook demonstrates how to build and fine-tune an LSTM model for time series forecasting.

**Steps:**
1. Generate synthetic dataset
2. Prepare data for supervised learning
3. Build and train LSTM
4. Fine-tune with lower learning rate
5. Evaluate and visualize results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import os

os.makedirs("../visuals", exist_ok=True)
np.random.seed(42)

In [ ]:
# Generate synthetic time series
time = np.arange(200)
series = 0.05 * time + 2*np.sin(0.1*time) + np.random.normal(0,0.2,200)

plt.plot(time, series)
plt.title("Synthetic Time Series")
plt.show()

In [ ]:
# Prepare supervised data
def create_dataset(series, window_size=10):
    X, y = [], []
    for i in range(len(series)-window_size):
        X.append(series[i:i+window_size])
        y.append(series[i+window_size])
    return np.array(X), np.array(y)

scaler = MinMaxScaler()
series_scaled = scaler.fit_transform(series.reshape(-1,1)).flatten()
X, y = create_dataset(series_scaled, window_size=10)
X = X.reshape((X.shape[0], X.shape[1], 1))

X_train, X_test = X[:150], X[150:]
y_train, y_test = y[:150], y[150:]

In [ ]:
# Build LSTM model
model = Sequential([
    LSTM(50, activation='tanh', input_shape=(10,1)),
    Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse')
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Fine-tune with lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
history_fine = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Plot training vs validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.plot(range(20,30), history_fine.history['loss'], label='Fine-tune Train Loss')
plt.plot(range(20,30), history_fine.history['val_loss'], label='Fine-tune Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('LSTM Training & Fine-Tuning')
plt.savefig("../visuals/training_loss.png")
plt.show()

In [ ]:
# Evaluate on test set
preds = model.predict(X_test)
preds_rescaled = scaler.inverse_transform(preds)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1,1))

plt.figure(figsize=(8,4))
plt.plot(range(len(y_test_rescaled)), y_test_rescaled, label="Actual")
plt.plot(range(len(preds_rescaled)), preds_rescaled, label="Forecast")
plt.legend(); plt.title("Forecast vs Actual (LSTM)")
plt.savefig("../visuals/forecast_vs_actual.png")
plt.show()